<a href="https://colab.research.google.com/github/thabang-r13/fraud-detection/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'creditcardfraud:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F310%2F23498%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240830%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240830T161745Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2e884c82b6d794cee7086f8cf199f9c2e3c53ae067ad0570e2a869abf0b2a1201362940bcfceaf52f3017c0cc089a55be5b32ad709416cb7dca9c8fc4e9c742e78b33be0f5df67fae6bcdda672d963d1cd152d16ddd9ccb54d089eadcd20ff020aa8db25805db6a8048227396d10242b7d6440db5e0a4a7eabaf3a428099bb466c132c7278266f5c2d17887fcef0d55468f3614ded0619c9bd6e99ef4c55f9889000c9928701d7ec35062b1681facca97b6abe92c8326016d176adafd17a55701db3c856dd6e221c9196df123ded3bc202b5422c556489b923bcbf8a90e97eff63c0c6cd03fa111dbcea241e54bf56a3b090f8417ba56dd9f414e5e0aba71f17'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 69155672 bytes downloaded
Downloaded and uncompressed: creditcardfraud
Data source import complete.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from tensorflow import keras
from tensorflow.keras import layers

In [5]:
creditcard_data = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

In [9]:
#Removing missing data
creditcard_data.dropna()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [11]:
model = keras.Sequential([
    layers.BatchNormalization(),

    layers.Dense(units = 2, activation = 'relu'),
    layers.Dropout(0.2),

    layers.Dense(units = 1, activation = 'sigmoid'),
    layers.Dropout(0.2)
]
)

In [12]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

In [14]:
data = creditcard_data[['Time', 'Amount']]
labels = creditcard_data['Class']

In [15]:
history = model.fit(data, labels, epochs = 10, validation_split = 0.8)

Epoch 1/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - binary_accuracy: 0.7884 - loss: 0.3286 - val_binary_accuracy: 0.9985 - val_loss: 0.0255
Epoch 2/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - binary_accuracy: 0.9975 - loss: 0.0594 - val_binary_accuracy: 0.9985 - val_loss: 0.0178
Epoch 3/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - binary_accuracy: 0.9971 - loss: 0.0393 - val_binary_accuracy: 0.9985 - val_loss: 0.0131
Epoch 4/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 23s 7ms/step - binary_accuracy: 0.9972 - loss: 0.0293 - val_binary_accuracy: 0.9985 - val_loss: 0.0115
Epoch 5/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - binary_accuracy: 0.9968 - loss: 0.0306 - val_binary_accuracy: 0.9985 - val_loss: 0.0115
Epoch 6/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - binary_accuracy: 0.9971 - loss: 0.0272 - val_binary_accuracy: 0.9985 - val_loss: 0.0111
Epoch 7/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - binary_accuracy: 0.9972 - loss: 0.0271 - val_binary_accuracy: 0.9985 - va

In [16]:
history_df = pd.DataFrame(history.history)

print('Best validation loss : ' + str(history_df['val_loss'].min()))
print('Best Accuray  : ' + str(history_df['binary_accuracy'].max()))

Best validation loss : 0.01104981079697609
Best Accuray  : 0.9972437024116516
